# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298257096376                   -0.85    5.4
  2   -8.300174365242       -2.72       -1.25    1.4
  3   -8.300445900427       -3.57       -1.88    3.1
  4   -8.300462556623       -4.78       -2.74    1.4
  5   -8.300464352174       -5.75       -3.07    5.8
  6   -8.300464488701       -6.86       -3.21    1.2
  7   -8.300464568026       -7.10       -3.36    8.0
  8   -8.300464601808       -7.47       -3.47    1.0
  9   -8.300464635642       -7.47       -3.70    1.1
 10   -8.300464635674      -10.50       -3.73    1.8
 11   -8.300464642073       -8.19       -3.96    1.0
 12   -8.300464643940       -8.73       -4.56    1.1


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64144645130                   -0.70    5.2
  2   -16.67792342130       -1.44       -1.14    1.4
  3   -16.67922401072       -2.89       -1.88    4.2
  4   -16.67928040920       -4.25       -2.74    3.2
  5   -16.67928598707       -5.25       -3.10    5.5
  6   -16.67928620964       -6.65       -3.52    1.6
  7   -16.67928622063       -7.96       -4.01    1.8


In [5]:
self_consistent_field(aluminium_setup(4); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32648079956                   -0.56    6.8
  2   -33.33420778826       -2.11       -1.00    1.6
  3   -33.33597306865       -2.75       -1.72    4.6
  4   -33.33613351317       -3.79       -2.63    4.6
  5   -33.33677716950       -3.19       -2.21    9.8
  6   -33.33681536488       -4.42       -2.27    1.2
  7   -33.33694375639       -3.89       -3.51    2.8
  8   -33.33694388495       -6.89       -3.74    5.4
  9   -33.33694378331   +   -6.99       -3.76    2.0
 10   -33.33694392575       -6.85       -4.74    2.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298613331218                   -0.85    5.0
  2   -8.300280408257       -2.78       -1.58    1.2
  3   -8.300387681665       -3.97       -2.28    5.4
  4   -8.300313424619   +   -4.13       -2.16    5.5
  5   -8.300464276862       -3.82       -3.60    1.6
  6   -8.300464605182       -6.48       -3.98    3.9
  7   -8.300464640841       -7.45       -4.50    2.1


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32560612419                   -0.56    7.2
  2   -33.30853970327   +   -1.77       -1.26    1.0
  3   -9.795836780647   +    1.37       -0.38    8.1
  4   -33.33400688725        1.37       -1.96    7.0
  5   -33.06065306540   +   -0.56       -1.24    5.4
  6   -32.50437099209   +   -0.25       -1.10    4.8
  7   -33.32990900731       -0.08       -2.06    5.0
  8   -33.33635019351       -2.19       -2.32    3.8
  9   -33.33672759583       -3.42       -2.64    2.1
 10   -33.33687624078       -3.83       -2.93    2.2
 11   -33.33690609474       -4.52       -3.16    1.9
 12   -33.33694323397       -4.43       -3.73    2.9
 13   -33.33694292715   +   -6.51       -3.79    4.4
 14   -33.33694382621       -6.05       -4.17    2.4


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.